In [ ]:
import tensorflow as tf
STRATEGY = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
NUM_DEVICES = STRATEGY.num_replicas_in_sync

from notebook_utils import create_model
from notebook_utils import calculate_flops

## Creating a model that can be transferred to a siking model

In [ ]:
model_name = "vgg16" #Model architecture (supported: vgg16 / alexnet)
model = create_model(STRATEGY, model_name=model_name, input_shape=(224,224,3), num_classes=10)

## Estimating the model flops 

In [ ]:
flops = calculate_flops(model)